In [3]:
from sklearn.tree import DecisionTreeRegressor
import mlflow
import os

In [4]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("arboles")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1713174288421, experiment_id='1', last_update_time=1713174288421, lifecycle_stage='active', name='arboles', tags={}>

# Con Hiperparámetros

In [15]:
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import preprocesamiento_datos

X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)
clf = DecisionTreeRegressor(criterion="absolute_error", max_depth=10, max_features=None, max_leaf_nodes=80, min_samples_leaf=10, min_samples_split=30, splitter="best", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado en el conjunto de validación
y_pred_val = clf.predict(X_val)

# Predecir con el modelo entrenado en el conjunto de prueba
y_pred_test = clf.predict(X_test)

# Calcular el error en el conjunto de validación
error_val = mean_absolute_error(y_val, y_pred_val)
print("Árbol de decisión - Validación:", error_val)

# Calcular el error en el conjunto de prueba
error_test = mean_absolute_error(y_test, y_pred_test)
print("Árbol de decisión - Prueba:", error_test)

# Crear DataFrame de predicciones vs. reales para el conjunto de validación
pred_vs_real_val = pd.DataFrame({'y_val': y_val, 'y_pred_val': y_pred_val, 'dif_val': abs(y_val - y_pred_val)})

# Crear DataFrame de predicciones vs. reales para el conjunto de prueba
pred_vs_real_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test, 'dif_test': abs(y_test - y_pred_test)})

# Mostrar los DataFrames
print("Predicciones vs. reales - Validación:")
print(pred_vs_real_val)

print("Predicciones vs. reales - Prueba:")
print(pred_vs_real_test)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="FINAL MODEL TREE") as run:
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("max_features", None)
    mlflow.log_param("max_leaf_nodes", 80)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("min_samples_split", 12)
    mlflow.log_param("splitter", "random")
    # Registrar la métrica de error en el conjunto de validación
    mlflow.log_metric("mean_absolute_error_val", error_val)
    # Registrar la métrica de error en el conjunto de prueba
    mlflow.log_metric("mean_absolute_error_test", error_test)
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar los DataFrames como un artefacto
    pred_vs_real_val.to_csv("pred_vs_real_val.csv", index=False)
    mlflow.log_artifact("pred_vs_real_val.csv")
    
    pred_vs_real_test.to_csv("pred_vs_real_test.csv", index=False)
    mlflow.log_artifact("pred_vs_real_test.csv")
# Obtener la importancia de las variables
importancias_variables = clf.feature_importances_

# Crear un DataFrame para almacenar la importancia de las variables junto con sus nombres
importancias_df = pd.DataFrame({'Variable': X_train.columns,
                               'Importancia': importancias_variables})

# Ordenar el DataFrame por importancia en orden descendente
importancias_df = importancias_df.sort_values(by='Importancia', ascending=False)

# Mostrar las 20 variables más importantes
print("Las 20 variables más importantes:")
print(importancias_df.head(40))

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Árbol de decisión - Validación: 3066171.477079796
Árbol de decisión - Prueba: 2851779.359430605
Predicciones vs. reales - Validación:
           y_val  y_pred_val     dif_val
1100    200000.0   1500000.0   1300000.0
1101  30000000.0  16000000.0  14000000.0
1102  30000000.0  40000000.0  10000000.0
1103   6000000.0   8000000.0   2000000.0
1104  40000000.0  25000000.0  15000000.0
...          ...         ...         ...
1684   1000000.0   1500000.0    500000.0
1685   1000000.0    700000.0    300000.0
1686    800000.0    800000.0         0.0
1687    500000.0    325000.0    175000.0
1688    500000.0   1000000.0    500000.0

[589 rows x 3 columns]
Predicciones vs. reales - Prueba:
          y_test  y_pred_test    dif_test
1689  25000000.0   25000000.0         0.0
1690  25000000.0   20000000.0   5000000.0
1691  30000000.0   25000000.0   5000000.0
1692    300000.0    

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Sin hiperparámetros

In [6]:
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import preprocesamiento_datos

X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)
clf = DecisionTreeRegressor(criterion="absolute_error", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado en el conjunto de validación
y_pred_val = clf.predict(X_val)

# Predecir con el modelo entrenado en el conjunto de prueba
y_pred_test = clf.predict(X_test)

# Calcular el error en el conjunto de validación
error_val = mean_absolute_error(y_val, y_pred_val)
print("Árbol de decisión - Validación:", error_val)

# Calcular el error en el conjunto de prueba
error_test = mean_absolute_error(y_test, y_pred_test)
print("Árbol de decisión - Prueba:", error_test)

# Crear DataFrame de predicciones vs. reales para el conjunto de validación
pred_vs_real_val = pd.DataFrame({'y_val': y_val, 'y_pred_val': y_pred_val, 'dif_val': abs(y_val - y_pred_val)})

# Crear DataFrame de predicciones vs. reales para el conjunto de prueba
pred_vs_real_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test, 'dif_test': abs(y_test - y_pred_test)})

# Mostrar los DataFrames
print("Predicciones vs. reales - Validación:")
print(pred_vs_real_val)

print("Predicciones vs. reales - Prueba:")
print(pred_vs_real_test)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="MODEL Sin HiperParámetros") as run:
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("max_features", None)
    mlflow.log_param("max_leaf_nodes", 80)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("min_samples_split", 12)
    mlflow.log_param("splitter", "random")
    # Registrar la métrica de error en el conjunto de validación
    mlflow.log_metric("mean_absolute_error_val", error_val)
    # Registrar la métrica de error en el conjunto de prueba
    mlflow.log_metric("mean_absolute_error_test", error_test)
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar los DataFrames como un artefacto
    pred_vs_real_val.to_csv("pred_vs_real_val.csv", index=False)
    mlflow.log_artifact("pred_vs_real_val.csv")
    
    pred_vs_real_test.to_csv("pred_vs_real_test.csv", index=False)
    mlflow.log_artifact("pred_vs_real_test.csv")

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Árbol de decisión - Validación: 3664983.0220713075
Árbol de decisión - Prueba: 3806628.1138790036
Predicciones vs. reales - Validación:
           y_val  y_pred_val     dif_val
1100    200000.0    600000.0    400000.0
1101  30000000.0  25000000.0   5000000.0
1102  30000000.0  25000000.0   5000000.0
1103   6000000.0  20000000.0  14000000.0
1104  40000000.0  28000000.0  12000000.0
...          ...         ...         ...
1684   1000000.0   5000000.0   4000000.0
1685   1000000.0    150000.0    850000.0
1686    800000.0    400000.0    400000.0
1687    500000.0    600000.0    100000.0
1688    500000.0    600000.0    100000.0

[589 rows x 3 columns]
Predicciones vs. reales - Prueba:
          y_test  y_pred_test    dif_test
1689  25000000.0   32000000.0   7000000.0
1690  25000000.0   24000000.0   1000000.0
1691  30000000.0   40000000.0  10000000.0
1692    300000.0  

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
